# 11.3 Web Scraping

**HTML**, which stands for "hypertext markup language", is an XML-like language for specifying the appearance of web pages. Each tag in HTML corresponds to a specific page element. For example:

- `<img>` specifies an image. The path to the image file is specified in the `src=` attribute.
- `<a>` specifies a hyperlink. The text enclosed between `<a>` and `</a>` is the text of the link that appears, while the URL is specified in the `href=` attribute of the tag.
- `<table>` specifies a table. The rows of the table are specified by `<tr>` tags nested inside the `<table>` tag, while the cells in each row are specified by `<td>` tags nested inside each `<tr>` tag.

Our goal in this section is not to teach you HTML to make a web page. You will learn just enough HTML to be able to scrape data programmatically from a web page.

# Inspecting HTML Source Code

Suppose we want to scrape faculty information from the [100 Best Films Website](https://www.hollywoodreporter.com/lists/100-best-films-ever-hollywood-favorites-818512) (`https://www.hollywoodreporter.com/lists/100-best-films-ever-hollywood-favorites-818512`). Once we have identified a web page that we want to scrape, the next step is to study the HTML source code. All web browsers have a "View Source" or "Page Source" feature that will display the HTML source of a web page. 

Visit the web page above, and view the HTML source of that page. (You may have to search online to figure out how to view the page source in your favorite browser.) Scroll down until you find the HTML code for the table containing information about the name, office, phone, e-mail, and office hours of the faculty members.

Notice how difficult it can be to find a page element in the HTML source. Many browsers allow you to right-click on a page element and jump to the part of the HTML source corresponding to that element.

# Web Scraping Using BeautifulSoup

`BeautifulSoup` is a Python library that makes it easy to navigate an HTML document. Like with `lxml`, we can query tags by name or attribute, and we can narrow our search to the ancestors and descendants of specific tags. In fact, it is possible to use `lxml` with HTML documents, but many web sites have malformed HTML, and `lxml` is not very forgiving. `BeautifulSoup` handles malformed HTML more gracefully and is thus the library of choice.

First, we issue an HTTP request to the URL to get the HTML source code.

In [34]:
import requests
response = requests.get("https://www.hollywoodreporter.com/lists/100-best-films-ever-hollywood-favorites-818512")

The HTML source is stored in the `.content` attribute of the response object. We pass this HTML source into `BeautifulSoup` to obtain a tree-like representation of the HTML document.

In [35]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

Now we can search for tags within this HTML document, using tags like `.find_all()`. For example, we can find all headings on this page.

In [36]:
headings = soup.find_all("header")
len(headings)


102

There is one movie per heading except the first heading item can be ignored. We iterate over all rows, extracting information about each movie to append to `rows`, which we will eventually turn into a `DataFrame`. As you read the code below, refer to the HTML source above, so that you understand what each line is doing. In particular, ask yourself the following questions:

You are encouraged to add `print()` statements inside the `for` loop to check your understanding of each line of code.

As a visual inspection of [the web page](https://statistics.calpoly.edu/content/StatisticsDirectory%26Office%20Hours) would confirm, there are 2 tables on the page, and we are interested in the second one.

In [37]:
rows = []
for movie in headings[1:]:
    # Get all the cells in the row.
    movieTitle = movie.find("h1").text
    try :
        movieYear = movie.find("h2").text
    except:
        movieYear = None
    # Append this data.
    rows.append({
        "title": movieTitle,
        "year": movieYear
    })
    
print (rows[:5]) # print first 5 rows

[{'title': "Hollywood's 100 Favorite Films", 'year': None}, {'title': 'The Seven Samurai', 'year': '(1954)'}, {'title': 'Bonnie and Clyde', 'year': '(1967)'}, {'title': 'Reservoir Dogs', 'year': '(1992)'}, {'title': 'Airplane!', 'year': '(1980)'}]


In the code above, observe that `.find_all()` returns a list with all matching tags, while `.find()` returns only the first matching tag. If no matching tags are found, then `.find_all()` will return an empty list `[]`, while `.find()` will return `None`.

Finally, we turn `rows` into a `DataFrame`.

In [38]:
import pandas as pd
pd.DataFrame(rows)

,title,year
0,Hollywood's 100 Favorite Films,None
1,The Seven Samurai,(1954)
2,Bonnie and Clyde,(1967)
3,Reservoir Dogs,(1992)
4,Airplane!,(1980)
...,...,...
96,Pulp Fiction,(1994)
97,The Shawshank Redemption,(1994)
98,Citizen Kane,(1941)
99,The Wizard of Oz,(1939)


Now this data is ready for further processing.

# Ethical Enlightenment: `robots.txt`

Web robots are crawling the web all the time. A website may want to restrict the robots from crawling specific pages. One reason is financial: each visit to a web page, by a human or a robot, costs the website money, and the website may prefer to save their limited bandwidth for human visitors. Another reason is privacy: a website may not want a search engine to preserve a snapshot of a page for all eternity.

To specify what a web robot is and isn't allowed to crawl, most websites will place a text file named `robots.txt` in the top-level directory of the web server. For example, the Statistics department web page has a `robots.txt` file: https://statistics.calpoly.edu/robots.txt

The format of the `robots.txt` file should be self-explanatory, but you can read a full specification of the standard here: http://www.robotstxt.org/robotstxt.html. As you scrape websites using your web robot, always check the `robots.txt` file first, to make sure that you are respecting the wishes of the website owner.

# Exercises

Find an easy website to scrape. What do I mean by easy?? Well, the CSE UCR Faculty website is developed in such a way to deter scrappers. Click on the link and checkout the source code. Can you understand it?? https://www1.cs.ucr.edu/people/faculty/

Find another website and extract some elements from that website. Try the following websites as a suggestion:
1. https://www.imdb.com/list/ls050274118/
2. https://www.imdb.com/list/ls000055475/
3. https://www.forbes.com/best-states-for-business/list/

In [1]:
#exercise activity
#used https://www.dataquest.io/blog/web-scraping-beautifulsoup/ for reference.
from requests import get
url = 'https://www.imdb.com/list/ls050274118/'
response = get(url)
from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)
movie_containers = html_soup.find_all('div', class_ = 'lister-item mode-detail')
print(type(movie_containers))
print(len(movie_containers))
#up to here then modified previous provided code.
rows = []
for movie2 in movie_containers[0:]: #need 0 to get first actor
    # Get all the cells in the row.
    place = movie2.find("h3").span.text
    try :
        actorname = movie2.find("h3").a.text
    except:
        actorname = None
    try :
        moviename = movie2.find("p").a.text
    except:
        moviename = None
    # Append this data.
    rows.append({
        "place": place,
        "actor": actorname.replace('\n',''),
        "movie": moviename.replace('\n','')
    })
    
print (rows[:5])
import pandas as pd
pd.DataFrame(rows)

<class 'bs4.element.ResultSet'>
100
[{'place': '1. ', 'actor': ' Jack Nicholson', 'movie': ' Chinatown'}, {'place': '2. ', 'actor': ' Marlon Brando', 'movie': ' Apocalypse Now'}, {'place': '3. ', 'actor': ' Robert De Niro', 'movie': ' Raging Bull'}, {'place': '4. ', 'actor': ' Al Pacino', 'movie': ' Serpico'}, {'place': '5. ', 'actor': ' Daniel Day-Lewis', 'movie': ' There Will Be Blood'}]


,place,actor,movie
0,1.,Jack Nicholson,Chinatown
1,2.,Marlon Brando,Apocalypse Now
2,3.,Robert De Niro,Raging Bull
3,4.,Al Pacino,Serpico
4,5.,Daniel Day-Lewis,There Will Be Blood
...,...,...,...
95,96.,Alan Rickman,Die Hard
96,97.,Edward G. Robinson,Double Indemnity
97,98.,Will Smith,The Fresh Prince of Bel-Air
98,99.,John Goodman,10 Cloverfield Lane


In [66]:
#exercise activity
#used https://www.dataquest.io/blog/web-scraping-beautifulsoup/ for reference.
from requests import get
url = 'https://www.imdb.com/list/ls000055475/'
response = get(url)
from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)
movie_containers = html_soup.find_all('div', class_ = 'lister-item mode-detail')
print(type(movie_containers))
print(len(movie_containers))
#up to here then modified previous provided code.
rows = []
for movie2 in movie_containers[0:]: #need 0 to get first actor
    # Get all the cells in the row.
    place = movie2.find("h3").span.text
    try :
        actorname = movie2.find("h3").a.text
    except:
        actorname = None
    try :
        moviename = movie2.find("p").a.text
    except:
        moviename = None
    # Append this data.
    rows.append({
        "rank": place,
        "actor": actorname.replace('\n',''),
        "movie": moviename.replace('\n','')
    })
    
print (rows[:5])
import pandas as pd
pd.DataFrame(rows)

<class 'bs4.element.ResultSet'>
100
[{'rank': '1. ', 'actor': ' Meryl Streep', 'movie': ' The Devil Wears Prada'}, {'rank': '2. ', 'actor': ' Ingrid Bergman', 'movie': ' Casablanca'}, {'rank': '3. ', 'actor': ' Vivien Leigh', 'movie': ' A Streetcar Named Desire'}, {'rank': '4. ', 'actor': ' Bette Davis', 'movie': ' All About Eve'}, {'rank': '5. ', 'actor': ' Jodie Foster', 'movie': ' The Silence of the Lambs'}]


,rank,actor,movie
0,1.,Meryl Streep,The Devil Wears Prada
1,2.,Ingrid Bergman,Casablanca
2,3.,Vivien Leigh,A Streetcar Named Desire
3,4.,Bette Davis,All About Eve
4,5.,Jodie Foster,The Silence of the Lambs
...,...,...,...
95,96.,Amanda Seyfried,Les Misérables
96,97.,Sophie Marceau,La boum
97,98.,Victoria Abril,¡Átame!
98,99.,Giulietta Masina,Le notti di Cabiria


In [98]:
#exercise activity
#used https://www.dataquest.io/blog/web-scraping-beautifulsoup/ (for reference.)
#used https://pythonprogramming.net/tables-xml-scraping-parsing-beautiful-soup-tutorial/ (for reference.)
from requests import get
url = 'https://www.forbes.com/best-states-for-business/list/'
response = get(url)
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')
type(soup)
#business = html_soup.findAll('table')[0].findAll('tr')
tables = soup.find('table')
table_rows = soup.find_all('tr')
print(table_rows)

[<tr>
<th></th>
<th data-field="position"><a>Rank</a></th>
<th data-field="name"><a>State</a></th>
<th data-field="businessCost"><a>Business<br/>Costs<br/>Rank</a></th>
<th data-field="laborSupply"><a>Labor<br/>Supply<br/>Rank</a></th>
<th data-field="regulatoryEnvironment"><a>Regulatory<br/>Environment<br/>Rank</a></th>
<th data-field="economicClimate"><a>Economic<br/>Climate<br/>Rank</a></th>
<th data-field="growthProspects"><a>Growth<br/>Prospects<br/>Rank</a></th>
<th data-field="lifeQuality"><a>Quality of<br/>Life<br/>Rank</a></th>
<th data-field="population"><a>Population</a></th>
</tr>, <tr class="data">
<td class="image"><a href="[LINK]"><img alt="" data-src="https://thumbor.forbes.com/thumbor/100x100/https://i.forbesimg.com/media/lists/[IMAGE_TYPE]/[IMAGE_URI]_416x416.jpg"/></a></td>
<td class="rank">[RANK]</td>
<td class="name"><a href="[LINK]">[NAME]</a></td>
<td>[BUSINESS_COST]</td>
<td>[LABOR_SUPPLY]</td>
<td>[REGULATORY_ENVIRONMENT]</td>
<td>[ECONOMIC_CLIMATE]</td>
<td>[G